In [1]:
# !pip install SQLAlchemy
# !pip install pyspark
# !pip install jupysql duckdb duckdb-engine

In [2]:
import pandas as pd
import pyspark as psk
from pyspark.sql import SparkSession
import time as t  
from pyspark.sql.types import *
from pyspark.sql import functions as f
from pyspark.sql import Row
from functools import reduce
from pyspark.sql.types import StructType, StructField, StringType
from datetime import date, datetime, timedelta
from joblib import Parallel, delayed
import multiprocessing
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame
import numpy as np
from math import modf
import sqlite3
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark import SparkConf, SQLContext

spark = SparkSession.builder.master("local").appName("Spark SQL").getOrCreate()
#both works
# 1: 
# spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")    #To resolve the error for p1075_38 to_timestamp formating: You may get a different result due to the upgrading to Spark >= 3.0: Fail to parse '1/2/2019 20:40:00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
# Set Spark SQL legacy time parser policy to LEGACY to handle older date formats
# 2:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
# Increase the max fields in the string representation of a plan
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)  # Increase to 1000 or more as needed


ps: /storage/icds/RISE/sw8/anaconda/anaconda3/envs/tensorflow/lib/libuuid.so.1: no version information available (required by /usr/lib64/libblkid.so.1)
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/16 04:50:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/16 04:50:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/16 04:50:41 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/03/16 04:50:41 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/03/16 04:50:41 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/03/16 04:50:41 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
25/03/16 04:50:41 WARN Utils: Service 'SparkUI' could not bind on port 4045. Att

In [3]:

# # Read CSV file into table

# duration_end_date = '2020-01-01'

# day_delta = timedelta(days = 1)
# split_date = duration_end_date.split('-')

# end_date = date(int(split_date[0]), int(split_date[1]), int(split_date[2]))
# start_date = end_date - timedelta(days = 90)
# print("start_date = {}, end_date = {}".format(start_date, end_date))

# df = spark.read.option("header",True).csv("P2457_faults.csv")
# # df.printSchema()
# # df.show()

# # display(df)

# # # DataFrame API Select query
# df.select("VIN","FAULT_DATE_TIME","FAULT_STATUS").show(20)

# # DataFrame API where()
# df.select("VIN","FAULT_DATE_TIME","FAULT_STATUS") \
#   .where("FAULT_STATUS == 'N'") \
#   .show(5)

# # df.select("VIN","FAULT_DATE_TIME",f.col('FAULT_DATE_TIME'), f.to_timestamp(f.col('FAULT_DATE_TIME'), 'yyyy-MM-dd HH:mm:ss'), f.to_date(f.col('FAULT_DATE_TIME'), 'yyyy-MM-dd')).show(5) 
# # \
# #   .where(f.to_timestamp(f.col('FAULT_DATE_TIME'), 'yyyy-MM-dd HH:mm:ss')> f.lit(start_date)) \
# #   .show(5)
# print("Filtering results:")  #for P1075_38: (f.to_timestamp(f.col('FAULT_DATE_TIME'), 'M/d/yyyy HH:mm') > start_date) & (f.to_timestamp(f.col('FAULT_DATE_TIME'), 'M/d/yyyy HH:mm') < end_date)
# df2 = df.filter((f.to_timestamp(f.col('FAULT_DATE_TIME'), 'yyyy-MM-dd HH:mm:ss') > start_date) & (f.to_timestamp(f.col('FAULT_DATE_TIME'), 'yyyy-MM-dd HH:mm:ss') < end_date))\
# .filter(f.col('FAULT_STATUS') == 'N')

# df2.select("VIN","FAULT_DATE_TIME","FAULT_STATUS").show(5)

# # # 

In [4]:
# Read CSV file into table

spark.read.option("header",True) \
          .csv("P1075_38.csv") \
          .createOrReplaceTempView("p1075_38")

spark.read.option("header",True) \
          .csv("P1075_75.csv") \
          .createOrReplaceTempView("p1075_75")

spark.read.option("header",True) \
          .csv("P1075_77.csv") \
          .createOrReplaceTempView("p1075_77")

spark.read.option("header",True) \
          .csv("P1075_86.csv") \
          .createOrReplaceTempView("p1075_86")

spark.read.option("header",True) \
          .csv("P1075_92.csv") \
          .createOrReplaceTempView("p1075_92")

spark.read.option("header",True) \
          .csv("P1075_94.csv") \
          .createOrReplaceTempView("p1075_94")

spark.read.option("header",True) \
          .csv("P0401_faults.csv") \
          .createOrReplaceTempView("p0401_faults")

spark.read.option("header",True) \
          .csv("P2457_faults.csv") \
          .createOrReplaceTempView("p2457_faults")

spark.read.option("header",True) \
          .csv("CCA Claims.csv") \
          .createOrReplaceTempView("cca_claims")

spark.read.option("header",True) \
          .csv("EGR Cooler Claims.csv") \
          .createOrReplaceTempView("egr_cooler_claims")

spark.read.option("header",True) \
          .csv("EGR FG 293 Claims.csv") \
          .createOrReplaceTempView("egr_fg_293_claims")

spark.read.option("header",True) \
          .csv("EGR Sensors.csv") \
          .createOrReplaceTempView("egr_sensors")

spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/PopulationWithChassisId.csv") \
          .createOrReplaceTempView("population")


In [5]:
# fault_date_time_format='M/d/yyyy'
fault_date_time_format = 'MM/dd/yyyy'

start_date='01/01/2016'
end_date='12/31/2021'

thisVIN = "1M1AN07Y8FM017857"
df_cca_claims = spark.sql(f"SELECT * FROM cca_claims WHERE VIN = '{thisVIN}'")
display(df_cca_claims)
df_egr_cooler_claims = spark.sql(f"SELECT * FROM egr_cooler_claims WHERE VIN = '{thisVIN}'")
df_egr_cooler_claims.show()
df_egr_fg_293_claims = spark.sql(f"SELECT * FROM egr_fg_293_claims WHERE VIN = '{thisVIN}'")
df_egr_fg_293_claims.show()
df_egr_sensors_claims = spark.sql(f"SELECT * FROM egr_sensors WHERE VIN = '{thisVIN}'")
df_egr_sensors_claims.show()

# Define filtering condition
filter_condition = (
    (f.to_timestamp(f.col('CLAIM_REG_DATE'), fault_date_time_format) > f.to_timestamp(f.lit(start_date), fault_date_time_format)) &
    (f.to_timestamp(f.col('CLAIM_REG_DATE'), fault_date_time_format) < f.to_timestamp(f.lit(end_date), fault_date_time_format))

)

# Filter dataframes based on the condition
df_cca_claims_part_replacements = df_cca_claims.filter(filter_condition)
df_egr_cooler_claims_part_replacements = df_egr_cooler_claims.filter(filter_condition)
df_egr_fg_293_claims_part_replacements = df_egr_fg_293_claims.filter(filter_condition)
df_egr_sensors_claims_part_replacements = df_egr_sensors_claims.filter(filter_condition)

# Count replacements in each table
total_replacements = (
    df_cca_claims_part_replacements.count() +
    df_egr_cooler_claims_part_replacements.count() +
    df_egr_fg_293_claims_part_replacements.count() +
    df_egr_sensors_claims_part_replacements.count()
)

if total_replacements > 0:
    print("YES")
else:
    print("NO")


DataFrame[BUSINESS_UNIT: string, CAMP_TYPE: string, CLAIM_HANDLING_DECISION: string, CLAIM_JOB_ID: string, CLAIM_REG_DATE: string, CLAIM_STATUS_DETAIL: string, CRED_DATE: string, DEB_CD: string, DEB_CD_DESC: string, DES_AREA: string, FGRP4: string, FGRP4_DESC: string, MAIN_OP: string, MAIN_OP_DESC: string, MILEAGE_MILES_CLAIM: string, PART_NAME: string, PART_NO: string, REF_NO: string, REPAIR_ID: string, REP_CNTRY: string, REP_DATE: string, REP_DEALER: string, REP_DEALER_NAME: string, REP_REGION: string, REP_STATE: string, SUPPL_NAME: string, SUPPL_NO: string, TOT_CLAIM_PAYMENT_USD: string, TOT_LABOR_PAYMENT_USD: string, TOT_MTR_PAYMENT_USD: string, TOT_OTHER_PAYMENT_USD: string, TOT_TRAVEL_PAYMENT_USD: string, VEH_AGE_CLAIM: string, VMT: string, WIS_CLAIM_ID: string, WIS_PRODUCT_ID: string, Veh Asmbly BUILT Mnth and Yr: string, APPL_TYPE: string, BRAND: string, CHASSIS_SERIES: string, CHASSIS_NO: string, COMPANY: string, CUST_NAME: string, CUST_NO: string, DEL_CNTRY: string, DEL_DEALE

+-------------+---------+-----------------------+------------------+--------------+-------------------+---------+------+-------------------+--------+-----+----------+-------------+-------------------+-------------------+--------------------+--------+------+--------------------+---------+---------+----------+---------------+----------+---------+--------------------+--------+---------------------+---------------------+-------------------+---------------------+----------------------+-------------+---+------------+--------------+--------------------+---------+-----+--------------+----------+-------+--------------------+----------+---------+-----------+---------------+----------+---------+----------+------------------+---------+---------+-----------+----------+------------------------+---------+---------+-------+---------------+---------------+-------------+--------+-----------------+--------------------+--------------------+--------------------+--------------------+
|BUSINESS_UNIT|CAMP_TYP

+-------------+---------+-----------------------+------------------+--------------+-------------------+---------+------+-------------------+--------+-----+----------+-------------+-------------------+-------------------+--------------------+--------+------+--------------------+---------+---------+----------+---------------+----------+---------+--------------------+--------+---------------------+---------------------+-------------------+---------------------+----------------------+-------------+---+------------+--------------+--------------------+---------+-----+--------------+----------+-------+--------------------+----------+---------+-----------+---------------+----------+---------+----------+------------------+---------+---------+-----------+----------+------------------------+---------+---------+-------+---------------+---------------+-------------+--------+-----------------+--------------------+--------------------+--------------------+--------------------+
|BUSINESS_UNIT|CAMP_TYP

+-------------+---------+-----------------------+------------+--------------+-------------------+---------+------+-----------+--------+-----+----------+-------+------------+-------------------+---------+-------+------+---------+---------+--------+----------+---------------+----------+---------+----------+--------+---------------------+---------------------+-------------------+---------------------+----------------------+-------------+---+------------+--------------+--------------------+---------+-----+--------------+----------+-------+---------+-------+---------+----------+---------------+----------+---------+----------+------------------+---------+---------+-----------+--------+------------------------+---------+--------+-------+---------------+---------------+-------------+--------+---+----------+----------+----------+--------------------+
|BUSINESS_UNIT|CAMP_TYPE|CLAIM_HANDLING_DECISION|CLAIM_JOB_ID|CLAIM_REG_DATE|CLAIM_STATUS_DETAIL|CRED_DATE|DEB_CD|DEB_CD_DESC|DES_AREA|FGRP4|FGRP4

In [ ]:
import pandas as pd
from pyspark.sql import functions as f
from pyspark.sql.utils import AnalysisException

# Path to output CSV file
output_csv_path = "/storage/home/yqf5148/work/volvoPennState/data/dataset/all_VINs_part_replacements.csv"

# Initialize an empty list to store the results
replacement_records = []

# Read VINs data
# path_to_VINs_data_part_repl = "/storage/home/yqf5148/work/volvoPennState/data/dataset/VINs_data.csv"
# VINs_their_part_repl_is_checked = pd.read_csv(path_to_VINs_data_part_repl, header=None, names=['VIN'], index_col=False, dtype='unicode')


df_all_pop_VINs = spark.sql('select distinct VIN from population ') 
df_pop_VINs_p = df_all_pop_VINs.toPandas()
print(df_all_pop_VINs.count())


# Set Spark SQL legacy time parser policy to LEGACY to handle older date formats
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

for index, row in df_pop_VINs_p.iterrows():
    thisVIN = row['VIN']
    # thisVIN = "1M1AN07Y8FM017857"
    print(f"Processing VIN: {thisVIN}")

    fault_date_time_format = 'MM/dd/yyyy'
    start_date = '2014-12-31'
    end_date = '2021-12-31'

    # Load all claims tables for the specific VIN
    claims_datasets = {
        'cca_claims': f"SELECT VIN, CLAIM_REG_DATE, TOT_CLAIM_PAYMENT_USD FROM cca_claims WHERE VIN = '{thisVIN}'",
        'egr_cooler_claims': f"SELECT VIN, CLAIM_REG_DATE, TOT_CLAIM_PAYMENT_USD FROM egr_cooler_claims WHERE VIN = '{thisVIN}'",
        'egr_fg_293_claims': f"SELECT VIN, CLAIM_REG_DATE, TOT_CLAIM_PAYMENT_USD FROM egr_fg_293_claims WHERE VIN = '{thisVIN}'",
        'egr_sensors_claims': f"SELECT VIN, CLAIM_REG_DATE, TOT_CLAIM_PAYMENT_USD FROM egr_sensors WHERE VIN = '{thisVIN}'"
    }

    # Define filtering condition with corrected date format
    for dataset_name, query in claims_datasets.items():
        try:
            df_claims = spark.sql(query)

            if df_claims is not None and df_claims.count() > 0:
                # print("hhssss")

                # df_claims.printSchema()  # Debugging step
                # df_claims.show(5, truncate=False)  # Show sample data

                df_claims = df_claims.withColumn("CLAIM_REG_DATE", f.to_date(f.col("CLAIM_REG_DATE"), "MM/dd/yyyy"))

                # Try alternative filtering
                df_filtered = df_claims.filter(
                    (f.col('CLAIM_REG_DATE') >= f.to_date(f.lit(start_date), "yyyy-MM-dd")) &
                    (f.col('CLAIM_REG_DATE') <= f.to_date(f.lit(end_date), "yyyy-MM-dd"))
                )

                # df_filtered.show(5, truncate=False)  # Show filtered data

                if df_filtered is not None and df_filtered.count() > 0:
                    # print("kooo")
                    for row in df_filtered.collect():
                        replacement_records.append([thisVIN, dataset_name, row['CLAIM_REG_DATE'], row['TOT_CLAIM_PAYMENT_USD']])

        except AnalysisException as e:
            print(f"Error processing dataset {dataset_name} for VIN {thisVIN}: {e}")

    # Convert list to DataFrame and save as CSV
    if replacement_records:
        df_output = pd.DataFrame(replacement_records, columns=['VIN', 'Dataset', 'Claim_Registration_Date', 'TOT_CLAIM_PAYMENT_USD'])
        df_output.to_csv(output_csv_path, index = None, mode = 'a', header=False) 
        print(f"CSV file saved at {output_csv_path}")
    else:
        print("No part replacements found within the specified period.")


322178
Processing VIN: 1M1AW01Y9FM007831
No part replacements found within the specified period.
Processing VIN: 1M2AV02C4FM013537
No part replacements found within the specified period.
Processing VIN: 1M2AV04C5GM013710
No part replacements found within the specified period.
Processing VIN: 1M1AW01X8GM008414
No part replacements found within the specified period.
Processing VIN: 4V4NC9EH1GN939854
CSV file saved at /storage/home/yqf5148/work/volvoPennState/data/dataset/all_VINs_part_replacements.csv
Processing VIN: 1M1AW01Y6GM008808
CSV file saved at /storage/home/yqf5148/work/volvoPennState/data/dataset/all_VINs_part_replacements.csv
Processing VIN: 1M1AW06X4GM008919
CSV file saved at /storage/home/yqf5148/work/volvoPennState/data/dataset/all_VINs_part_replacements.csv
Processing VIN: 1M1AW01Y1GM009025
CSV file saved at /storage/home/yqf5148/work/volvoPennState/data/dataset/all_VINs_part_replacements.csv
Processing VIN: 1M1AW01X1HM009423
CSV file saved at /storage/home/yqf5148/work/vo

CSV file saved at /storage/home/yqf5148/work/volvoPennState/data/dataset/all_VINs_part_replacements.csv
Processing VIN: 4V4NC9EH9GN953713
CSV file saved at /storage/home/yqf5148/work/volvoPennState/data/dataset/all_VINs_part_replacements.csv
Processing VIN: 1M1AW02Y6FM050781
CSV file saved at /storage/home/yqf5148/work/volvoPennState/data/dataset/all_VINs_part_replacements.csv
Processing VIN: 4V4NC9EH6GN953717
CSV file saved at /storage/home/yqf5148/work/volvoPennState/data/dataset/all_VINs_part_replacements.csv
Processing VIN: 1M1AW09Y1FM050788
CSV file saved at /storage/home/yqf5148/work/volvoPennState/data/dataset/all_VINs_part_replacements.csv
Processing VIN: 4V4NC9EH2GN953732
CSV file saved at /storage/home/yqf5148/work/volvoPennState/data/dataset/all_VINs_part_replacements.csv
Processing VIN: 4V4NC9EH1GN953818
CSV file saved at /storage/home/yqf5148/work/volvoPennState/data/dataset/all_VINs_part_replacements.csv
Processing VIN: 1M1AW07Y8FM050905
CSV file saved at /storage/home/yq

CSV file saved at /storage/home/yqf5148/work/volvoPennState/data/dataset/all_VINs_part_replacements.csv
Processing VIN: 1M2AV04C5HM014664
CSV file saved at /storage/home/yqf5148/work/volvoPennState/data/dataset/all_VINs_part_replacements.csv
Processing VIN: 1M2AV02C2GM014719
CSV file saved at /storage/home/yqf5148/work/volvoPennState/data/dataset/all_VINs_part_replacements.csv
Processing VIN: 1M1AW01X2HM009558
CSV file saved at /storage/home/yqf5148/work/volvoPennState/data/dataset/all_VINs_part_replacements.csv
Processing VIN: 1M1AW01X2HM009656
CSV file saved at /storage/home/yqf5148/work/volvoPennState/data/dataset/all_VINs_part_replacements.csv
Processing VIN: 1M1AW01X7HM009734
CSV file saved at /storage/home/yqf5148/work/volvoPennState/data/dataset/all_VINs_part_replacements.csv
Processing VIN: 1M2AV02C9GM014927
CSV file saved at /storage/home/yqf5148/work/volvoPennState/data/dataset/all_VINs_part_replacements.csv
Processing VIN: 1M1AW01X4HM009836
CSV file saved at /storage/home/yq